In [101]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [79]:
tickers = ["MSFT", "AMZN", "^GSPC", "^DJI"]
data = yf.download(tickers, start="2015-01-01", end="2025-01-01", auto_adjust=True)

[*********************100%***********************]  4 of 4 completed


## Research Data

In [67]:
def compute_table1(df):
    df['Year'] = df.index.year
    df['Direction'] = np.where(df['Close'].diff() > 0, 'Increase', 'Decrease')
    yearly_counts = df.groupby("Year")["Direction"].value_counts().unstack(fill_value=0)
    for col in ["Increase", "Decrease"]:
        if col not in yearly_counts:
            yearly_counts[col] = 0
    yearly_counts["Total"] = yearly_counts["Increase"] + yearly_counts["Decrease"]
    yearly_counts["Increase %"] = (yearly_counts["Increase"] / yearly_counts["Total"] * 100).round(2)
    yearly_counts["Decrease %"] = (yearly_counts["Decrease"] / yearly_counts["Total"] * 100).round(2)
    yearly_summary = yearly_counts[["Increase", "Increase %", "Decrease", "Decrease %", "Total"]]

    total_row = pd.DataFrame(yearly_summary.sum(numeric_only=True)).T
    total_row.index = ["Total"]
    total_row["Increase %"] = (total_row["Increase"] / total_row["Total"] * 100).round(2)
    total_row["Decrease %"] = (total_row["Decrease"] / total_row["Total"] * 100).round(2)

    final_table = pd.concat([yearly_summary, total_row])
    final_table.reset_index(inplace=True)
    final_table.rename(columns={'index': 'Year'}, inplace=True)

    return final_table

In [69]:
def compute_table2(df):
    rows = []
    for year, group in df.groupby("Year"):
        n = len(group)
        if n < 2: continue
        mid = n // 2
        train, hold = group.iloc[:mid], group.iloc[mid:]
        
        def count_dir(subset):
            inc = (subset["Direction"] == "Increase").sum()
            dec = (subset["Direction"] == "Decrease").sum()
            total = inc + dec
            return inc, dec, total
        
        inc_t, dec_t, total_t = count_dir(train)
        inc_h, dec_h, total_h = count_dir(hold)
        rows.append([year, inc_t, dec_t, total_t, inc_h, dec_h, total_h])
    
    df2 = pd.DataFrame(rows, columns=["Year", "Train Increase", "Train Decrease", "Train Total",
                                      "Hold Increase", "Hold Decrease", "Hold Total"])
    
    total_row = pd.DataFrame([["Total", df2["Train Increase"].sum(), df2["Train Decrease"].sum(),
                               df2["Train Total"].sum(), df2["Hold Increase"].sum(),
                               df2["Hold Decrease"].sum(), df2["Hold Total"].sum()]],
                             columns=df2.columns)
    df2 = pd.concat([df2, total_row], ignore_index=True)
    return df2

In [71]:
def compute_table3(df):
    rows = []
    for year, group in df.groupby("Year"):
        n = len(group)
        if n < 2: continue
        mid = n // 2
        train, hold = group.iloc[:mid], group.iloc[mid:]
        
        def count_dir(subset):
            inc = (subset["Direction"] == "Increase").sum()
            dec = (subset["Direction"] == "Decrease").sum()
            total = inc + dec
            return inc, dec, total
        
        inc_t, dec_t, total_t = count_dir(train)
        inc_h, dec_h, total_h = count_dir(hold)
        rows.append([year, inc_t, dec_t, total_t, inc_h, dec_h, total_h])
    
    df3 = pd.DataFrame(rows, columns=["Year", "Train Increase", "Train Decrease", "Train Total",
                                      "Hold Increase", "Hold Decrease", "Hold Total"])
    
    total_row = pd.DataFrame([["Total", df3["Train Increase"].sum(), df3["Train Decrease"].sum(),
                               df3["Train Total"].sum(), df3["Hold Increase"].sum(),
                               df3["Hold Decrease"].sum(), df3["Hold Total"].sum()]],
                             columns=df3.columns)
    df3 = pd.concat([df3, total_row], ignore_index=True)
    return df3

In [77]:
for ticker, df in ohlcv.items():

    
    print(f"\n{'='*70}")
    print(f"{ticker} - Table 1: Yearly Increase/Decrease percentages")
    print(f"\n{'='*70}")
    table1 = compute_table1(df)
    display(table1)

    print(f"\n{'='*70}")
    print(f" {ticker} - Table 2: Parameter selection data (20% split)")
    print(f"\n{'='*70}")
    table2 = compute_table2(df)
    display(table2)

    print(f"\n{'='*70}")
    print(f" {ticker} - Table 3: Comparison Data (50% train / 50% holdout)")
    print(f"\n{'='*70}")
    table3 = compute_table3(df)
    display(table3)



MSFT - Table 1: Yearly Increase/Decrease percentages



/var/folders/g2/rncyb9xn1fs6qs0fjnbpl7j00000gn/T/ipykernel_4009/2505137587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df.index.year
/var/folders/g2/rncyb9xn1fs6qs0fjnbpl7j00000gn/T/ipykernel_4009/2505137587.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Direction'] = np.where(df['Close'].diff() > 0, 'Increase', 'Decrease')


Direction,Year,Increase,Increase %,Decrease,Decrease %,Total
0,2015,123.0,48.81,129.0,51.19,252.0
1,2016,130.0,51.59,122.0,48.41,252.0
2,2017,136.0,54.18,115.0,45.82,251.0
3,2018,138.0,54.98,113.0,45.02,251.0
4,2019,148.0,58.73,104.0,41.27,252.0
5,2020,143.0,56.52,110.0,43.48,253.0
6,2021,132.0,52.38,120.0,47.62,252.0
7,2022,115.0,45.82,136.0,54.18,251.0
8,2023,137.0,54.80,113.0,45.20,250.0
9,2024,135.0,53.57,117.0,46.43,252.0



 MSFT - Table 2: Parameter selection data (20% split)



,Year,Train Increase,Train Decrease,Train Total,Hold Increase,Hold Decrease,Hold Total
0,2015,52,74,126,71,55,126
1,2016,61,65,126,69,57,126
2,2017,64,61,125,72,54,126
3,2018,69,56,125,69,57,126
4,2019,78,48,126,70,56,126
5,2020,74,52,126,69,58,127
6,2021,67,59,126,65,61,126
7,2022,60,65,125,55,71,126
8,2023,64,61,125,73,52,125
9,2024,68,58,126,67,59,126


...

Direction,Year,Increase,Increase %,Decrease,Decrease %,Total
0,2015,121.0,48.02,131.0,51.98,252.0
1,2016,136.0,53.97,116.0,46.03,252.0
2,2017,144.0,57.37,107.0,42.63,251.0
3,2018,134.0,53.39,117.0,46.61,251.0
4,2019,145.0,57.54,107.0,42.46,252.0
5,2020,140.0,55.34,113.0,44.66,253.0
6,2021,140.0,55.56,112.0,44.44,252.0
7,2022,121.0,48.21,130.0,51.79,251.0
8,2023,139.0,55.60,111.0,44.40,250.0
9,2024,139.0,55.16,113.0,44.84,252.0



 ^DJI - Table 2: Parameter selection data (20% split)



,Year,Train Increase,Train Decrease,Train Total,Hold Increase,Hold Decrease,Hold Total
0,2015,62,64,126,59,67,126
1,2016,70,56,126,66,60,126
2,2017,61,64,125,83,43,126
3,2018,67,58,125,67,59,126
4,2019,71,55,126,74,52,126
5,2020,65,61,126,75,52,127
6,2021,73,53,126,67,59,126
7,2022,58,67,125,63,63,126
8,2023,64,61,125,75,50,125
9,2024,72,54,126,67,59,126



 ^DJI - Table 3: Comparison Data (50% train / 50% holdout)



,Year,Train Increase,Train Decrease,Train Total,Hold Increase,Hold Decrease,Hold Total
0,2015,62,64,126,59,67,126
1,2016,70,56,126,66,60,126
2,2017,61,64,125,83,43,126
3,2018,67,58,125,67,59,126
4,2019,71,55,126,74,52,126
5,2020,65,61,126,75,52,127
6,2021,73,53,126,67,59,126
7,2022,58,67,125,63,63,126
8,2023,64,61,125,75,50,125
9,2024,72,54,126,67,59,126


## Summary Statistics for selected indicators 

In [115]:
results = {}

for ticker in tickers:
    indicators = continuous_data_features(ohlcv[ticker])
    results[ticker] = indicators

summary_stats = {}

for ticker, indicators in results.items():
    summary = indicators.describe().loc[["mean", "std", "min", "max"]]
    summary_stats[ticker] = summary

for ticker, stats in summary_stats.items():
    print(f"\n{ticker}")
    print(stats.round(2))


MSFT
       SMA10   WMA10  MOM10  STOCHK  STOCHD  RSI14   MACD   WILLR   ADOSC  \
mean  187.52  187.75   1.55   61.03   61.05  55.79   1.08  -38.97   40.69   
std   122.45  122.57  10.08   29.04   26.60  15.56   3.26   29.04   10.11   
min    41.08   41.03 -47.79    0.00    2.01   6.71 -11.83 -100.00    0.00   
max   459.86  461.60  39.86  100.00   98.73  99.11  11.53   -0.00  100.00   

       CCI20  
mean   28.60  
std    85.03  
min  -351.29  
max   374.53  

AMZN
       SMA10   WMA10  MOM10  STOCHK  STOCHD  RSI14   MACD   WILLR   ADOSC  \
mean  101.17  101.29   0.84   59.56   59.57  55.50   0.56  -40.44   53.70   
std    53.82   53.87   7.14   29.71   27.60  16.25   2.37   29.71   11.06   
min    14.69   14.61 -37.67    0.00    1.28  11.44 -12.43 -100.00    0.00   
max   227.36  228.76  28.97  100.00   98.78  93.98   8.33   -0.00  100.00   

       CCI20  
mean   26.15  
std    86.33  
min  -276.02  
max   458.76  

^GSPC
        SMA10    WMA10   MOM10  STOCHK  STOCHD  RSI14    MA

## Create Continuous Data features 

In [113]:
def continuous_data_features(df):
    indicators = pd.DataFrame(index=df.index)

    close = df["Close"]
    high = df["High"]
    low = df["Low"]
    vol = df["Volume"]

    # Simple Moving Average 10 day
    indicators["SMA10"] = close.rolling(window=10).mean()

    # Weighted Moving Average 10 day
    weights = np.arange(1, 11)
    indicators["WMA10"] = close.rolling(10).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)

    # Momentum 10 day
    indicators["MOM10"] = close - close.shift(10)

    # Stochastic Oscillator %K 14 day
    indicators["STOCHK"] = 100 * (close - low.rolling(14).min()) / (high.rolling(14).max() - low.rolling(14).min())

    # Stochastic Oscillator %D 3 day SMA of %K
    indicators["STOCHD"] = indicators["STOCHK"].rolling(3).mean()

    # Relative Strength Index 14 day 
    delta = close.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    rs = up.rolling(14).mean() / down.rolling(14).mean()
    indicators["RSI14"] = 100 - (100 / (1 + rs))

    # MACD 12-day EMA - 26 day EMA
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    indicators["MACD"] = ema12 - ema26

    # Williams %R 14 day
    highest14 = high.rolling(14).max()
    lowest14 = low.rolling(14).min()
    indicators["WILLR"] = -100 * (highest14 - close) / (highest14 - lowest14)

    # 9. Accumulation/Distribution Oscillator (ADOSC)
    clv = np.where((high - low) == 0, 0, ((close - low) - (high - close)) / (high - low))
    adl = (clv * vol).cumsum()

    ema3_adl = adl.ewm(span=3, adjust=False).mean()
    ema10_adl = adl.ewm(span=10, adjust=False).mean()

    adosc = ema3_adl - ema10_adl

    # Scale ADOSC to 0–100 like in papers
    min_val = adosc.min()
    max_val = adosc.max()
    indicators["ADOSC"] = 100 * (adosc - min_val) / (max_val - min_val)


    # 10. Commodity Channel Index 20 day
    tp = (high + low + close) / 3
    sma = tp.rolling(20).mean()
    mad = (tp - sma).abs().rolling(20).mean()
    indicators["CCI20"] = (tp - sma) / (0.015 * mad)

    return indicators


## Create Trend Deterministic Data Features 

In [29]:
def trend_deterministic_data(indicators, df):
    trend = pd.DataFrame(index=indicators.index)

    close = df["Adj Close"]
    high = df["High"]
    low = df["Low"]
    vol = df["Volume"]

    #SMA
    trend["SMA10_T"] = np.where(close > indicators["SMA10"], 1, -1)

    #WMA
    trend["WMA10_T"] = np.where(close > indicators["WMA10"], 1, -1)

    #Momentum
    trend["MOM10_T"] = np.where(indicators["MOM10"] > 0, 1, -1)

    # Sophisticated Oscillator trends
    trend["STOCHK_T"] = np.where(indicators["STOCHK"] > indicators["STOCHK"].shift(1), 1, -1)
    trend["STOCHD_T"] = np.where(indicators["STOCHD"] > indicators["STOCHD"].shift(1), 1, -1)

    #RSI trend
    rsi = indicators["RSI14"]
    trend["RSI14_T"] = np.select([rsi > 70, rsi < 30, rsi > rsi.shift(1)],
                                 [-1, 1, 1], default=-1)
    #MACD trend
    trend["MACD_T"] = np.where(indicators["MACD"] > indicators["MACD"].shift(1), 1, -1)

    #Williams %R trend
    trend["WILLR_T"] = np.where(indicators["WILLR"] > indicators["WILLR"].shift(1), 1, -1)

    #ADOSC trend
    trend["ADOSC_T"] = np.where(indicators["ADOSC"] > indicators["ADOSC"].shift(1), 1, -1)

    #CCI trend
    cci = indicators["CCI20"]
    trend["CCI20_T"] = np.select([cci > 200, cci < -200, cci > cci.shift(1)],
                                 [-1, 1, 1], default=-1)
    return trend
    

In [31]:
trend_tables = {}

for ticker in tickers:
    df = ohlcv[ticker]
    indicators = continuous_data_features(df)
    trend_tables[ticker] = trend_deterministic_data(indicators, df)

trend_tables["AMZN"].tail(100)
for ticker, df in trend_tables.items():
    print(f"\n{ticker} unique values:")
    print(df.nunique())  # number of unique values per column
    print("Unique values per column:")
    for col in df.columns:
        print(col, df[col].unique()[:5])  # show example unique values


MSFT unique values:
SMA10_T     2
WMA10_T     2
MOM10_T     2
STOCHK_T    2
STOCHD_T    2
RSI14_T     2
MACD_T      2
WILLR_T     2
ADOSC_T     2
CCI20_T     2
dtype: int64
Unique values per column:
SMA10_T [-1  1]
WMA10_T [-1  1]
MOM10_T [-1  1]
STOCHK_T [-1  1]
STOCHD_T [-1  1]
RSI14_T [-1  1]
MACD_T [-1  1]
WILLR_T [-1  1]
ADOSC_T [-1  1]
CCI20_T [-1  1]

AMZN unique values:
SMA10_T     2
WMA10_T     2
MOM10_T     2
STOCHK_T    2
STOCHD_T    2
RSI14_T     2
MACD_T      2
WILLR_T     2
ADOSC_T     2
CCI20_T     2
dtype: int64
Unique values per column:
SMA10_T [-1  1]
WMA10_T [-1  1]
MOM10_T [-1  1]
STOCHK_T [-1  1]
STOCHD_T [-1  1]
RSI14_T [-1  1]
MACD_T [-1  1]
WILLR_T [-1  1]
ADOSC_T [-1  1]
CCI20_T [-1  1]

^GSPC unique values:
SMA10_T     2
WMA10_T     2
MOM10_T     2
STOCHK_T    2
STOCHD_T    2
RSI14_T     2
MACD_T      2
WILLR_T     2
ADOSC_T     2
CCI20_T     2
dtype: int64
Unique values per column:
SMA10_T [-1  1]
WMA10_T [-1  1]
MOM10_T [-1  1]
STOCHK_T [-1  1]
STOCHD_T [-1